## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [2]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dmitriair/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

In [4]:
with open("wordsim_similarity_goldstandard.txt", encoding="utf-8") as rf:
  triples = [line.strip().split("\t") for line in rf.readlines()]
  score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples}

Отметим, что из исходного набора данных мы взяли только экспертные оценки сходства (similarity) и только для существительных. Исходный набор данных доступен по [ссылке](http://alfonseca.org/pubs/ws353simrel.tar.gz)

Посмотрим на примеры оценок. 

У слов может быть по несколько значений, которые различаются в WordNet. Здесь -- ради примера -- мы будем "жадно" выбирать первое попавшееся, но далее будем работать с ними иначе.




In [13]:
for w1, w2 in list(score_map)[:2]:
  
  print("\nWords: %s-%s\nGround truth score: %.2f" % (w1, w2, score_map[(w1, w2)]))
  
  ss1 = wn.synset(w1 + ".n.01")
  ss2 = wn.synset(w2 + ".n.01")

  print("\nPath: %.3f" % ss1.path_similarity(ss2), end=" ")
  print("\nwup: %.3f" % ss1.wup_similarity(ss2), end=" ")
  print("\nshortest_path: %.3f" % ss1.shortest_path_distance(ss2))


Words: tiger-cat
Ground truth score: 7.35

Path: 0.091 
wup: 0.545 
shortest_path: 10.000

Words: tiger-tiger
Ground truth score: 10.00

Path: 1.000 
wup: 0.750 
shortest_path: 0.000


Вычисляем для всех пар несколько оценок

In [7]:
from itertools import product

list_pairs = list(score_map)
wup_list, true_list, path_list = [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)
    
    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

## Вычисляем ранговую корреляцию Спирмена

In [12]:
from scipy.stats import spearmanr

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.4f" % coef)

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.4f" % coef)


wup  Spearman R: 0.6438
path Spearman R: 0.6176
